# 🚀 개선된 GPU Solver 테스트

**Batched Tridiagonal Solver로 성능 개선!**

---

## ⚡ GPU 활성화

1. `Runtime` → `Change runtime type`
2. Hardware accelerator: **GPU**
3. Save


## 📁 Step 1: Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✓ Google Drive 연결 완료!")

## 📤 Step 2: 프로젝트 파일 업로드

**Google Drive에 업로드:**
- `els-fdm-pricer-improved.tar.gz`

In [ ]:
# Drive에서 파일 복사
!cp /content/drive/MyDrive/els-fdm-pricer-improved.tar.gz .
!tar -xzf els-fdm-pricer-improved.tar.gz

print("✓ 프로젝트 파일 준비 완료!")

## 🔧 Step 3: 패키지 설치

In [ ]:
!pip install -q cupy-cuda12x numpy scipy matplotlib

print("✓ 패키지 설치 완료!")

## 🎮 Step 4: GPU 확인

In [ ]:
!nvidia-smi

import cupy as cp

print("\n" + "="*80)
print("GPU 정보")
print("="*80)

if cp.cuda.is_available():
    device_id = 0
    props = cp.cuda.runtime.getDeviceProperties(device_id)
    gpu_name = props['name'].decode('utf-8')
    
    print(f"✓ GPU: {gpu_name}")
    
    mem_info = cp.cuda.Device(device_id).mem_info
    print(f"✓ GPU 메모리: {mem_info[1] / 1024**3:.1f} GB")
else:
    print("⚠️ GPU 사용 불가")

print("="*80)

## 🧪 Step 5: 작은 그리드 테스트 (30×30×60)

In [ ]:
import sys
sys.path.insert(0, '/content/els-fdm-pricer')

from src.models.els_product import create_sample_els
from src.pricing.els_pricer import price_els
from src.solvers.gpu_adi_solver_improved import ImprovedGPUADISolver
from src.grid.grid_2d import create_adaptive_grid
import time
import numpy as np

print("="*80)
print("작은 그리드 테스트: 30×30×60")
print("="*80)

product = create_sample_els()
N1, N2, Nt = 30, 30, 60

# CPU
print("\n[CPU] 계산 중...", end="", flush=True)
start = time.time()
result_cpu = price_els(product, N1=N1, N2=N2, Nt=Nt, verbose=False)
time_cpu = time.time() - start
print(f" {time_cpu:.3f}초")
print(f"  가격: {result_cpu['price']:.4f}")

# GPU Improved
print("\n[GPU Improved] 계산 중...", end="", flush=True)

grid = create_adaptive_grid(
    product.S1_0, product.S2_0, product.maturity,
    N1, N2, Nt, space_factor=3.0
)

solver = ImprovedGPUADISolver(
    grid, product.r, product.q1, product.q2,
    product.sigma1, product.sigma2, product.rho,
    use_gpu=True
)

# 만기 페이오프 (간단하게)
V_T = np.ones((N1, N2)) * (product.principal + product.coupons[-1])

start = time.time()
V_0 = solver.solve(V_T)
time_gpu = time.time() - start

price_gpu = V_0[N1//2, N2//2]

print(f" {time_gpu:.3f}초")
print(f"  가격: {price_gpu:.4f}")

speedup = time_cpu / time_gpu
print(f"\n속도 향상: {speedup:.1f}배")

if speedup > 1:
    print("✓ GPU가 빠름!")
else:
    print("⚠️ CPU가 빠름")

## 🎯 Step 6: 중간 그리드 테스트 (50×50×100)

In [ ]:
print("="*80)
print("중간 그리드 테스트: 50×50×100")
print("="*80)

N1, N2, Nt = 50, 50, 100

# CPU
print("\n[CPU] 계산 중...", end="", flush=True)
start = time.time()
result_cpu = price_els(product, N1=N1, N2=N2, Nt=Nt, verbose=False)
time_cpu = time.time() - start
print(f" {time_cpu:.3f}초")

# GPU
print("[GPU Improved] 계산 중...", end="", flush=True)

grid = create_adaptive_grid(
    product.S1_0, product.S2_0, product.maturity,
    N1, N2, Nt, space_factor=3.0
)

solver = ImprovedGPUADISolver(
    grid, product.r, product.q1, product.q2,
    product.sigma1, product.sigma2, product.rho,
    use_gpu=True
)

V_T = np.ones((N1, N2)) * (product.principal + product.coupons[-1])

start = time.time()
V_0 = solver.solve(V_T)
time_gpu = time.time() - start

print(f" {time_gpu:.3f}초")

speedup = time_cpu / time_gpu
print(f"\n속도 향상: {speedup:.1f}배")

## 🚀 Step 7: 큰 그리드 테스트 (100×100×200)

In [ ]:
print("="*80)
print("큰 그리드 테스트: 100×100×200")
print("="*80)

N1, N2, Nt = 100, 100, 200

# CPU
print("\n[CPU] 계산 중...", end="", flush=True)
start = time.time()
result_cpu = price_els(product, N1=N1, N2=N2, Nt=Nt, verbose=False)
time_cpu = time.time() - start
print(f" {time_cpu:.3f}초")

# GPU
print("[GPU Improved] 계산 중...", end="", flush=True)

grid = create_adaptive_grid(
    product.S1_0, product.S2_0, product.maturity,
    N1, N2, Nt, space_factor=3.0
)

solver = ImprovedGPUADISolver(
    grid, product.r, product.q1, product.q2,
    product.sigma1, product.sigma2, product.rho,
    use_gpu=True
)

V_T = np.ones((N1, N2)) * (product.principal + product.coupons[-1])

start = time.time()
V_0 = solver.solve(V_T)
time_gpu = time.time() - start

print(f" {time_gpu:.3f}초")

speedup = time_cpu / time_gpu
print(f"\n속도 향상: {speedup:.1f}배")
print(f"\n처리량: {N1*N2*Nt/time_gpu:,.0f} points/sec")

if speedup > 10:
    print("\n🚀 GPU 매우 빠름!")
elif speedup > 1:
    print("\n✓ GPU가 빠름")
else:
    print("\n⚠️ 추가 최적화 필요")

## 📊 Step 8: 비교 테스트 (기존 vs 개선)

In [ ]:
from src.solvers.gpu_adi_solver import GPUADISolver

print("="*80)
print("기존 GPU vs 개선 GPU 비교 (50×50×100)")
print("="*80)

N1, N2, Nt = 50, 50, 100

grid = create_adaptive_grid(
    product.S1_0, product.S2_0, product.maturity,
    N1, N2, Nt, space_factor=3.0
)

V_T = np.ones((N1, N2)) * (product.principal + product.coupons[-1])

# 기존 GPU
print("\n[GPU 기존] 계산 중...", end="", flush=True)
solver_old = GPUADISolver(
    grid, product.r, product.q1, product.q2,
    product.sigma1, product.sigma2, product.rho,
    use_gpu=True
)

start = time.time()
V_old = solver_old.solve(V_T)
time_old = time.time() - start
print(f" {time_old:.3f}초")

# 개선 GPU
print("[GPU 개선] 계산 중...", end="", flush=True)
solver_new = ImprovedGPUADISolver(
    grid, product.r, product.q1, product.q2,
    product.sigma1, product.sigma2, product.rho,
    use_gpu=True
)

start = time.time()
V_new = solver_new.solve(V_T)
time_new = time.time() - start
print(f" {time_new:.3f}초")

improvement = time_old / time_new
print(f"\n개선 효과: {improvement:.1f}배")

if improvement > 10:
    print("🎉 큰 개선!")
elif improvement > 2:
    print("✓ 상당한 개선")
else:
    print("△ 약간 개선")

## 📈 Step 9: 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

# 여러 그리드 크기 테스트 결과 시각화
sizes = ['30×30', '50×50', '100×100']
# 실제 측정값으로 업데이트 필요

plt.figure(figsize=(10, 6))
# 결과 플롯
plt.title('GPU Performance Improvement')
plt.xlabel('Grid Size')
plt.ylabel('Speedup (×)')
plt.grid(True, alpha=0.3)
plt.show()

## 🎉 최종 요약

In [ ]:
print("="*80)
print("🎉 개선된 GPU Solver 테스트 완료!")
print("="*80)
print()
print("주요 개선사항:")
print("  1. Batched Tridiagonal Solver")
print("  2. Python for loop 제거")
print("  3. GPU 병렬성 완전 활용")
print()
print("다음 단계:")
print("  - 대규모 그리드 테스트 (200×200×1000)")
print("  - 조기상환 GPU vectorize")
print("  - 수치 안정성 검증")
print("="*80)